<a href="https://colab.research.google.com/github/hewness/EN.625.641.81.SU25-Final_Project/blob/main/EN_625_641_81_SU25_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objectives
At the end of this project you will be able to:

* Select stocks
* Analyze portfolio performance

# Instructions
Student will form groups of 2 (minimum) to 4 (maximum) students.

Your group has been given $1 million to build a portfolio with at least 4 stocks and at most 10 stocks.
You may pick any stocks you like but must select which stocks to include in your portfolio by the end of module 4.

# Deliverable
Write a concluding report about the overall performance of your portfolio.

These are the key performance indicators that you should highlight in your report:
1. Weekly rate of return and standard deviation of your portfolio and each of its components
1. Your portfolio Beta as well as its components Betas. (Please use the S&P 500 as the Market Portfolio).
1. Your portfolio Jensen index and Sharpe Ratio.
1. Your portfolio VaR (Value at Risk) throughout the month of July.

In [473]:
!pip install PyPortfolioOpt

In [474]:
import yfinance as yf
import pandas as pd
import plotly.express as px
import calendar
import numpy as np
import pypfopt

from google.colab import drive, data_table
from datetime import datetime, timedelta
from scipy.optimize import minimize, Bounds, LinearConstraint
from collections import OrderedDict
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models

data_table.enable_dataframe_formatter()

In [475]:
RANGE_SLIDER_VISIBLE = False
TARGET_RETURN = 0.08
RISK_FREE_RATE = 0.0435

In [476]:
# Define tickers
PORTFOLIO_TICKERS = ['MSFT', 'BEN', 'TSLA', 'KO', 'NVDA', 'GS', 'DASH', 'NFLX']
TICKERS = PORTFOLIO_TICKERS + ['SPY']

# Define the end and start date
END_DATE = '2025-06-30'
START_DATE = (datetime.strptime(END_DATE, '%Y-%m-%d') - timedelta(weeks=104)).strftime('%Y-%m-%d')


print(f"Start Date: {START_DATE}")
print(f"End Date: {END_DATE}")

Start Date: 2023-07-03
End Date: 2025-06-30


# Weekly Price Data

In [477]:
# Download weekly adjusted close prices
weekly_price_data_df = yf.download(TICKERS, start=START_DATE, end=END_DATE, interval='1wk', auto_adjust=False)['Adj Close']

[*********************100%***********************]  9 of 9 completed


In [478]:
weekly_price_data_df.head(5)

Ticker,BEN,DASH,GS,KO,MSFT,NFLX,NVDA,SPY,TSLA
Date,,,,,,,,,
2023-07-03,23.440563,77.949997,299.698395,56.259850,332.056641,438.100006,42.476860,427.179871,274.429993
2023-07-10,25.493296,83.959999,310.177429,57.333092,339.953827,441.910004,45.441040,437.631653,281.380005
2023-07-17,26.282116,84.500000,334.682343,58.782883,338.506378,427.500000,44.281746,440.456482,260.019989
2023-07-24,26.452429,89.650002,335.890015,58.820549,333.189026,425.779999,46.721249,445.073639,266.440002
2023-07-31,25.403656,83.599998,335.880524,57.154209,322.761200,431.600006,44.652515,435.225708,253.860001


In [479]:
fig = px.line(weekly_price_data_df, x=weekly_price_data_df.index, y=list(weekly_price_data_df.columns), labels={'value': 'Price', 'variable': 'Ticker'}, color='variable')
fig.update_xaxes(rangeslider_visible=RANGE_SLIDER_VISIBLE)
fig.show()

In [480]:
weekly_returns_df = weekly_price_data_df.pct_change().dropna()

In [481]:
weekly_returns_df.head()

Ticker,BEN,DASH,GS,KO,MSFT,NFLX,NVDA,SPY,TSLA
Date,,,,,,,,,
2023-07-10,0.087572,0.077101,0.034965,0.019077,0.023783,0.008697,0.069783,0.024467,0.025325
2023-07-17,0.030942,0.006432,0.079003,0.025287,-0.004258,-0.032608,-0.025512,0.006455,-0.075912
2023-07-24,0.006480,0.060947,0.003608,0.000641,-0.015708,-0.004023,0.055090,0.010483,0.024690
2023-07-31,-0.039648,-0.067485,-0.000028,-0.028329,-0.031297,0.013669,-0.044278,-0.022127,-0.047215
2023-08-07,-0.015173,-0.052751,-0.035360,0.007577,-0.020654,-0.023031,-0.085609,-0.002596,-0.044158


In [482]:
fig = px.line(weekly_returns_df, x=weekly_returns_df.index, y=list(weekly_returns_df.columns), labels={'value': 'Weekly Return', 'variable': 'Ticker'}, color='variable')
fig.update_xaxes(rangeslider_visible=RANGE_SLIDER_VISIBLE)
fig.show()

# Daily Price Data

In [483]:
# Our optimized portfolio is created by using two years of data from our portfolio start date
daily_price_data_df = yf.download(TICKERS, start=START_DATE, end=END_DATE, interval='1d', auto_adjust=False)['Adj Close']

historical_daily_start_date = datetime.strptime(START_DATE, '%Y-%m-%d') - timedelta(days=720)
historical_daily_end_date = START_DATE
historical_daily_price_data_df = yf.download(TICKERS, start=historical_daily_start_date, end=historical_daily_end_date, interval='1d', auto_adjust=False)['Adj Close']

[*********************100%***********************]  9 of 9 completed
[*********************100%***********************]  9 of 9 completed


In [484]:
daily_price_data_df.head()

Ticker,BEN,DASH,GS,KO,MSFT,NFLX,NVDA,SPY,TSLA
Date,,,,,,,,,
2023-07-03,24.103893,78.709999,310.576782,57.031826,332.814880,441.440002,42.386913,432.283997,279.820007
2023-07-05,23.781193,79.070000,304.338776,57.455463,332.972412,445.899994,42.290974,431.641144,282.480011
2023-07-06,23.324034,76.989998,297.634949,57.013004,336.044647,438.839996,42.077106,428.261078,276.540009
2023-07-07,23.440565,77.949997,299.698425,56.259850,332.056671,438.100006,42.476856,427.179871,274.429993
2023-07-10,24.336954,77.610001,300.934570,55.836205,326.749176,441.709991,42.154053,428.261078,269.609985


In [485]:
# add the missing mondays into the daily, forward fill those days
daily_price_data_df = daily_price_data_df.reindex(daily_price_data_df.index.union(weekly_price_data_df.index)).ffill()
data_table.DataTable(daily_price_data_df, max_columns=100, num_rows_per_page=30)

Ticker,BEN,DASH,GS,KO,MSFT,NFLX,NVDA,SPY,TSLA
Date,,,,,,,,,
2023-07-03,24.103893,78.709999,310.576782,57.031826,332.814880,441.440002,42.386913,432.283997,279.820007
2023-07-05,23.781193,79.070000,304.338776,57.455463,332.972412,445.899994,42.290974,431.641144,282.480011
2023-07-06,23.324034,76.989998,297.634949,57.013004,336.044647,438.839996,42.077106,428.261078,276.540009
2023-07-07,23.440565,77.949997,299.698425,56.259850,332.056671,438.100006,42.476856,427.179871,274.429993
2023-07-10,24.336954,77.610001,300.934570,55.836205,326.749176,441.709991,42.154053,428.261078,269.609985
...,...,...,...,...,...,...,...,...,...
2025-06-23,22.578041,230.320007,646.880005,69.739998,486.000000,1253.540039,144.169998,600.150024,348.679993
2025-06-24,22.913406,236.169998,662.109985,70.209999,490.109985,1279.109985,147.899994,606.780029,340.470001
2025-06-25,22.893679,232.679993,669.869995,69.629997,492.269989,1275.250000,154.309998,607.119995,327.549988


In [486]:
historical_daily_price_data_df.tail()

Ticker,BEN,DASH,GS,KO,MSFT,NFLX,NVDA,SPY,TSLA
Date,,,,,,,,,
2023-06-26,22.902231,73.580002,297.026306,57.634342,323.568665,415.940002,40.607006,420.254211,241.050003
2023-06-27,23.620146,74.760002,298.062805,57.511951,329.447266,417.079987,41.850243,424.861542,250.210007
2023-06-28,23.646734,76.489998,298.262482,56.975342,330.707672,429.839996,41.091721,425.075867,256.239990
2023-06-29,24.023216,75.449997,307.229614,56.504627,329.919861,428.239990,40.796890,426.751282,257.500000
2023-06-30,23.942541,76.419998,306.706635,56.692913,335.325806,440.489990,42.275978,431.787201,261.769989


In [487]:
daily_returns_df = daily_price_data_df.pct_change().dropna()
hisorical_daily_returns_df = historical_daily_price_data_df.pct_change().dropna()

In [488]:
daily_returns_df.head()

Ticker,BEN,DASH,GS,KO,MSFT,NFLX,NVDA,SPY,TSLA
Date,,,,,,,,,
2023-07-05,-0.013388,0.004574,-0.020085,0.007428,0.000473,0.010103,-0.002263,-0.001487,0.009506
2023-07-06,-0.019224,-0.026306,-0.022028,-0.007701,0.009227,-0.015833,-0.005057,-0.007831,-0.021028
2023-07-07,0.004996,0.012469,0.006933,-0.013210,-0.011867,-0.001686,0.009500,-0.002525,-0.007630
2023-07-10,0.038241,-0.004362,0.004125,-0.007530,-0.015984,0.008240,-0.007600,0.002531,-0.017564
2023-07-11,0.022099,0.028862,0.013935,0.003541,0.001929,-0.003396,0.005334,0.006369,0.000668


In [489]:
hisorical_daily_returns_df.head()

Ticker,BEN,DASH,GS,KO,MSFT,NFLX,NVDA,SPY,TSLA
Date,,,,,,,,,
2021-07-14,-0.015309,-0.040569,-0.004203,0.022537,0.005445,0.013446,-0.020173,0.001492,-0.022676
2021-07-15,-0.001323,-0.011811,-0.002804,0.003200,-0.005239,-0.009125,-0.044112,-0.003416,-0.004255
2021-07-16,-0.025836,0.000179,-0.022901,-0.000709,-0.000996,-0.023280,-0.042457,-0.007844,-0.009806
2021-07-19,-0.028222,0.048638,-0.027632,-0.011879,-0.013322,0.003715,0.034070,-0.014768,0.003104
2021-07-20,0.025892,0.011738,0.028304,0.001794,0.008339,-0.002311,-0.008932,0.014330,0.022098


In [490]:
rolling_7d_std_df = daily_returns_df.rolling('7D').std()
rolling_7d_std_df.head()

Ticker,BEN,DASH,GS,KO,MSFT,NFLX,NVDA,SPY,TSLA
Date,,,,,,,,,
2023-07-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07-06,0.004126,0.021835,0.001373,0.010698,0.006190,0.018340,0.001975,0.004486,0.021591
2023-07-07,0.012640,0.020491,0.016189,0.010686,0.010598,0.012986,0.007726,0.003403,0.015305
2023-07-10,0.025871,0.016743,0.015412,0.008856,0.011537,0.011879,0.007558,0.004268,0.013690
2023-07-11,0.024033,0.020457,0.016502,0.008619,0.010401,0.010413,0.007197,0.005365,0.012647


In [491]:
daily_returns_df.merge(rolling_7d_std_df, how='left', left_index=True, right_index=True, suffixes=('', '_7d_std')).head(7)

Ticker,BEN,DASH,GS,KO,MSFT,NFLX,NVDA,SPY,TSLA,BEN_7d_std,DASH_7d_std,GS_7d_std,KO_7d_std,MSFT_7d_std,NFLX_7d_std,NVDA_7d_std,SPY_7d_std,TSLA_7d_std
Date,,,,,,,,,,,,,,,,,,
2023-07-05,-0.013388,0.004574,-0.020085,0.007428,0.000473,0.010103,-0.002263,-0.001487,0.009506,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07-06,-0.019224,-0.026306,-0.022028,-0.007701,0.009227,-0.015833,-0.005057,-0.007831,-0.021028,0.004126,0.021835,0.001373,0.010698,0.006190,0.018340,0.001975,0.004486,0.021591
2023-07-07,0.004996,0.012469,0.006933,-0.013210,-0.011867,-0.001686,0.009500,-0.002525,-0.007630,0.012640,0.020491,0.016189,0.010686,0.010598,0.012986,0.007726,0.003403,0.015305
2023-07-10,0.038241,-0.004362,0.004125,-0.007530,-0.015984,0.008240,-0.007600,0.002531,-0.017564,0.025871,0.016743,0.015412,0.008856,0.011537,0.011879,0.007558,0.004268,0.013690
2023-07-11,0.022099,0.028862,0.013935,0.003541,0.001929,-0.003396,0.005334,0.006369,0.000668,0.024033,0.020457,0.016502,0.008619,0.010401,0.010413,0.007197,0.005365,0.012647
2023-07-12,0.024505,0.009267,0.017203,0.005376,0.014227,0.008723,0.035303,0.008046,0.008154,0.022070,0.020652,0.015484,0.007996,0.013099,0.010074,0.017083,0.006533,0.012207
2023-07-13,0.027436,0.033503,0.006985,0.008523,0.016192,0.014255,0.047264,0.007937,0.021729,0.012022,0.015364,0.005483,0.009262,0.014668,0.007499,0.022604,0.004503,0.015000


# Portfolio Optimization

In [492]:
def optimize_portfolio(prices, target_return, method='EF'):

    if method == 'EF':
        mu = pypfopt.expected_returns.mean_historical_return(prices)
        S = risk_models.sample_cov(prices)
        ef = EfficientFrontier(mu, S, weight_bounds=(0,0.30))
        # stuff to play around with
        weights = ef.max_sharpe()
        # weights = ef.efficient_return(target_return)
    else:
        returns_df = prices.pct_change().dropna()
        cov_daily_returns_df = returns_df.cov()

        ticker_count = returns_df.shape[1]
        initial_weights = np.ones(ticker_count) / ticker_count
        expected_returns = daily_returns_df[prices.columns].mean()

        constraints = [
            {
                'type': 'eq',
                'fun': lambda weights: np.sum(weights) - 1
            },
            {
                'type': 'ineq',
                'args': (expected_returns, target_return),
                'fun': lambda weights, expected_returns, target_return: np.dot(expected_returns.T, weights) - target_return,
            }
        ]
        portfolio_variance = lambda weights, cov_matrix: np.dot(weights.T, np.dot(cov_matrix, weights))

        result = minimize(
            portfolio_variance,
            initial_weights,
            args = (cov_daily_returns_df,),
            method='SLSQP',
            bounds=Bounds(0, 0.20),
            constraints=constraints
        )

        weights = OrderedDict(zip(returns_df.columns, result.x))

    return weights

In [493]:
optimize_portfolio(historical_daily_price_data_df[PORTFOLIO_TICKERS], TARGET_RETURN, method='EF')

OrderedDict([('MSFT', 0.3),
             ('BEN', 0.0),
             ('TSLA', 0.0240842886314248),
             ('KO', 0.3),
             ('NVDA', 0.3),
             ('GS', 0.0759157113685752),
             ('DASH', 0.0),
             ('NFLX', 0.0)])

In [494]:
optimize_portfolio(historical_daily_price_data_df[PORTFOLIO_TICKERS], TARGET_RETURN, method='SCIPY')

OrderedDict([('MSFT', np.float64(0.0)),
             ('BEN', np.float64(3.3819982260088272e-12)),
             ('TSLA', np.float64(0.19999999999952456)),
             ('KO', np.float64(1.6132322391394438e-13)),
             ('NVDA', np.float64(0.19999999999936263)),
             ('GS', np.float64(0.19999999999941556)),
             ('DASH', np.float64(0.199999999999436)),
             ('NFLX', np.float64(0.19999999999935855))])

# Backtesting

In [496]:
def compute_weighted_total_returns(returns_df, tickers):
    ticker_weight_names = [ticker + ' Open Weight' for ticker in tickers]

    returns_df['weighted_daily_return'] = 0
    returns_df['total_weight'] = 0

    returns = returns_df.to_numpy()
    time_series_length = len(returns[:,1])

    columns = list(returns_df.columns)

    weighted_values = list()

    for i in range(time_series_length):

        daily_return = 0

        # replace with weighted values for the open
        if weighted_values:

            for ticker_index, ticker in enumerate(tickers):
                ticker_weight_index = columns.index(ticker + ' Open Weight')
                returns[i, ticker_weight_index] = weighted_values[ticker_index]

            weighted_values = list()

        # iterating through each ticker
        # retrieving the open weights for each ticker
        # daily return = weight_1 * return_1 + weight_2 * return_2 + ... + weight_n * return_n
        # compute weight for next day for each ticker
        for ticker in tickers:
            ticker_index = columns.index(ticker)
            ticker_weight_index = columns.index(ticker + ' Open Weight')
            daily_return = daily_return + returns[i, ticker_index] * returns[i, ticker_weight_index]
            weighted_values.append((1 + returns[i, ticker_index]) * returns[i, ticker_weight_index])

        total_weight_weight_index = columns.index('total_weight')
        returns[i, total_weight_weight_index] = sum([returns[i, columns.index(ticker + ' Open Weight')] for ticker in tickers])

        # compute the new weights for the the next day (open)
        weighted_values = [ weighted_value / (1 + daily_return) for weighted_value in weighted_values]
        returns[i, columns.index('weighted_daily_return')] = daily_return

    return pd.DataFrame(returns, index=returns_df.index, columns=returns_df.columns)



def compute_cumulative_metrics(returns_df):
    columns = list(returns_df.columns)

    returns = returns_df.to_numpy()
    time_series_length = len(returns[:,1])
    product_return = 1

    for i in range(time_series_length):
        daily_return = returns[i, columns.index('weighted_daily_return')]
        product_return = (1 + daily_return) * product_return

        total_return_index = columns.index('total_return')
        unit_value_index = columns.index('unit_value')

        if i == 0:
            returns[i, unit_value_index] = 100 * (1 + daily_return)
        else:
            returns[i, unit_value_index] = returns[i-1, unit_value_index] * (1 + daily_return)

        returns[i, total_return_index] = product_return - 1

    cumulative_df = pd.DataFrame(returns, index=returns_df.index, columns=returns_df.columns)

    columns_std = TICKERS + ['weighted_daily_return']

    rolling_7d_std_df = cumulative_df[columns_std].rolling('7D').std()

    tickers_cov = TICKERS + ['weighted_daily_return']

    for ticker in tickers_cov:
        cov_column_name = ticker + '_SPY_7d_cov'
        cumulative_df[cov_column_name] = cumulative_df[ticker].rolling('7D').cov(cumulative_df['SPY'])

    cumulative_df = cumulative_df.merge(rolling_7d_std_df, how='left', left_index=True, right_index=True, suffixes=('', '_7d_std'))

    return cumulative_df


def compute_fund_metrics(returns_df, start_date, end_date, weights, rebalance_monthly=False):

    temp_start_date = start_date

    time_period_returns_df_list = list()

    allocation_df_list = list()

    while temp_start_date < end_date:
        temp_end_date = datetime(temp_start_date.year, temp_start_date.month, calendar.monthrange(temp_start_date.year, temp_start_date.month)[1])

        temp_start_date_str = temp_start_date.strftime('%Y-%m-%d')
        temp_end_date_str = temp_end_date.strftime('%Y-%m-%d') if rebalance_monthly else end_date.strftime('%Y-%m-%d')

        period_returns_df = returns_df.loc[temp_start_date_str:temp_end_date_str]
        period_returns_df = period_returns_df.copy(deep=True)

        period_returns_df['unit_value'] = 100
        period_returns_df['total_return'] = 0

        for ticker in TICKERS:
            period_returns_df[ticker + ' Open Weight'] = weights[ticker]

        period_returns_df = compute_weighted_total_returns(period_returns_df, TICKERS)
        time_period_returns_df_list.append(period_returns_df)

        if rebalance_monthly:
            temp_start_date = temp_start_date + pd.DateOffset(months=1)
        else:
            break

    daily_model_returns = pd.concat(time_period_returns_df_list)

    return compute_cumulative_metrics(daily_model_returns)

In [497]:
model_start_datetime = datetime.strptime(START_DATE, '%Y-%m-%d')
model_end_datetime = datetime.strptime(END_DATE, '%Y-%m-%d')

weights = optimize_portfolio(historical_daily_price_data_df, TARGET_RETURN, method='EF')
fund_results_df = compute_fund_metrics(daily_returns_df, model_start_datetime, model_end_datetime, weights)

data_table.DataTable(fund_results_df, max_columns=100, num_rows_per_page=30)

Ticker,BEN,DASH,GS,KO,MSFT,NFLX,NVDA,SPY,TSLA,unit_value,...,MSFT_7d_std,BEN_7d_std,TSLA_7d_std,KO_7d_std,NVDA_7d_std,GS_7d_std,DASH_7d_std,NFLX_7d_std,SPY_7d_std,weighted_daily_return_7d_std
Date,,,,,,,,,,,,,,,,,,,,,
2023-07-05,-0.013388,0.004574,-0.020085,0.007428,0.000473,0.010103,-0.002263,-0.001487,0.009506,100.154269,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07-06,-0.019224,-0.026306,-0.022028,-0.007701,0.009227,-0.015833,-0.005057,-0.007831,-0.021028,99.968899,...,0.006190,0.004126,0.021591,0.010698,0.001975,0.001373,0.021835,0.018340,0.004486,0.002400
2023-07-07,0.004996,0.012469,0.006933,-0.013210,-0.011867,-0.001686,0.009500,-0.002525,-0.007630,99.471163,...,0.010598,0.012640,0.015305,0.010686,0.007726,0.016189,0.020491,0.012986,0.003403,0.003262
2023-07-10,0.038241,-0.004362,0.004125,-0.007530,-0.015984,0.008240,-0.007600,0.002531,-0.017564,98.566440,...,0.011537,0.025871,0.013690,0.008856,0.007558,0.015412,0.016743,0.011879,0.004268,0.004532
2023-07-11,0.022099,0.028862,0.013935,0.003541,0.001929,-0.003396,0.005334,0.006369,0.000668,98.949487,...,0.010401,0.024033,0.012647,0.008619,0.007197,0.016502,0.020457,0.010413,0.005365,0.005157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-23,0.004388,0.044725,0.009488,0.013074,0.018014,0.017971,0.002224,0.009877,0.082319,196.414719,...,0.010552,0.013696,0.050506,0.011064,0.008794,0.009206,0.024222,0.009371,0.007655,0.007998
2025-06-24,0.014854,0.025399,0.023544,0.006739,0.008457,0.020398,0.025872,0.011047,-0.023546,199.825760,...,0.009898,0.004343,0.045344,0.009331,0.015445,0.006974,0.020565,0.008954,0.006838,0.010650
2025-06-25,-0.000861,-0.014778,0.011720,-0.008261,0.004407,-0.003018,0.043340,0.000560,-0.037948,204.260014,...,0.009910,0.006673,0.053725,0.010059,0.024298,0.006895,0.026541,0.010733,0.006680,0.013439


## Weekly Risk and Performance Measures Tabulated

In [500]:
fund_weekly_results_df = fund_results_df[fund_results_df.index.isin(daily_price_data_df.index)][['unit_value']]

fund_weekly_returns_results_df = fund_weekly_results_df[['unit_value']].pct_change().dropna()
fund_weekly_returns_results_df = fund_weekly_returns_results_df.rename(columns={'unit_value': 'PORTFOLIO'})

all_weekly_returns_df = weekly_returns_df.merge(fund_weekly_returns_results_df['PORTFOLIO'],how='left', left_index=True, right_index=True)

std_columns = [ticker + '_7d_std' for ticker in TICKERS]+ ['weighted_daily_return_7d_std']
cov_columns = [ticker + '_SPY_7d_cov' for ticker in TICKERS] + ['weighted_daily_return_SPY_7d_cov']
measures_columns = std_columns + cov_columns
all_weekly_df = all_weekly_returns_df.merge(fund_results_df[measures_columns], how='left', left_index=True, right_index=True)
all_weekly_df = all_weekly_df.rename(columns={'weighted_daily_return_7d_std': 'PORTFOLIO_7d_std',
                                              'weighted_daily_return_SPY_7d_cov': 'PORTFOLIO_SPY_7d_cov'})

risk_tickers = ['PORTFOLIO'] + TICKERS

for ticker in risk_tickers:
    all_weekly_df[ticker +'_sharpe'] = (all_weekly_df[ticker] - RISK_FREE_RATE) / all_weekly_df[ticker + '_7d_std']
    all_weekly_df[ticker +'_VaR_95'] = all_weekly_df[ticker] - 1.645 * all_weekly_df[ticker + '_7d_std']
    all_weekly_df[ticker +'_beta'] = all_weekly_df[ticker + '_SPY_7d_cov'] / (all_weekly_df['SPY_7d_std']**2)
    all_weekly_df[ticker +'_jensen'] = all_weekly_df[ticker] - (RISK_FREE_RATE + all_weekly_df[ticker + '_beta'] * (all_weekly_df['SPY'] - RISK_FREE_RATE))

data_table.DataTable(all_weekly_df, max_columns=100, num_rows_per_page=30)

,BEN,DASH,GS,KO,MSFT,NFLX,NVDA,SPY,TSLA,PORTFOLIO,...,DASH_beta,DASH_jensen,NFLX_sharpe,NFLX_VaR_95,NFLX_beta,NFLX_jensen,SPY_sharpe,SPY_VaR_95,SPY_beta,SPY_jensen
Date,,,,,,,,,,,,,,,,,,,,,
2023-07-10,0.087572,0.077101,0.034965,0.019077,0.023783,0.008697,0.069783,0.024467,0.025325,-0.009095,...,2.287063,0.077131,-2.929803,-0.010844,2.489047,0.012571,-4.459947,0.017447,1.0,-3.469447e-18
2023-07-17,0.030942,0.006432,0.079003,0.025287,-0.004258,-0.032608,-0.025512,0.006455,-0.075912,0.007137,...,1.711153,0.026322,-5.045630,-0.057422,2.617828,0.020869,-10.108912,0.000427,1.0,-1.387779e-17
2023-07-24,0.006480,0.060947,0.003608,0.000641,-0.015708,-0.004023,0.055090,0.010483,0.024690,0.003775,...,0.876061,0.046372,-0.938045,-0.087363,9.338793,0.260819,-6.214261,0.001743,1.0,-1.387779e-17
2023-07-31,-0.039648,-0.067485,-0.000028,-0.028329,-0.031297,0.013669,-0.044278,-0.022127,-0.047215,-0.004649,...,2.585903,0.058719,-1.216577,-0.026667,3.325833,0.188432,-11.161919,-0.031798,1.0,-2.775558e-17
2023-08-07,-0.015173,-0.052751,-0.035360,0.007577,-0.020654,-0.023031,-0.085609,-0.002596,-0.044158,0.010450,...,0.784740,-0.060078,-4.506881,-0.047314,1.812796,0.017032,-5.729447,-0.015831,1.0,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-26,0.006044,0.012127,0.003191,0.004598,0.022613,0.018424,0.029248,0.017751,0.020982,0.000000,...,-0.480715,-0.043751,-8.452455,0.013544,-0.266264,-0.031932,-3.641205,0.006119,1.0,1.249001e-16
2025-06-02,0.010629,0.046969,0.027663,-0.010402,0.021766,0.028362,0.048768,0.016543,-0.148127,0.009352,...,-0.680588,-0.014878,-0.886706,0.000280,0.731605,0.004585,-2.681892,0.000008,1.0,1.804112e-16
2025-06-09,0.013717,0.002335,-0.000749,-0.004625,0.009737,-0.023617,0.001764,-0.003572,0.102223,0.005622,...,1.334837,0.021668,-5.343782,-0.044278,-1.114972,-0.119601,-8.117690,-0.013111,1.0,8.118506e-16


In [501]:
all_weekly_df.mean()

,0
BEN,8.747188e-04
DASH,1.265674e-02
GS,8.959146e-03
KO,2.372850e-03
MSFT,4.353266e-03
...,...
NFLX_jensen,1.962822e-02
SPY_sharpe,-6.772917e+00
SPY_VaR_95,-9.517371e-03
SPY_beta,1.000000e+00


## Weekly Risk and Performance Measures Plotted

In [498]:
fig = px.line(fund_results_df['total_return'], x=fund_results_df.index, y='total_return', title='Portfolio Total Returns (' + START_DATE + ' to ' + END_DATE + ')')
fig.update_xaxes(rangeslider_visible=RANGE_SLIDER_VISIBLE)
fig.show()

In [499]:
fig = px.line(fund_results_df['weighted_daily_return'], x=fund_results_df.index, y='weighted_daily_return', title='Portfolio Weighted Daily Returns (' + START_DATE + ' to ' + END_DATE + ')')
fig.update_xaxes(rangeslider_visible=RANGE_SLIDER_VISIBLE)
fig.show()

In [502]:

def plot_measures(measure_label, data_df):
    fig = px.line(data_df,
                  x=data_df.index,
                  y=list(data_df.columns),
                labels = {  'value': measure_label,
                            'variable': 'Ticker'},
                color= 'variable',
                title= measure_label + ' (' + START_DATE + ' to ' + END_DATE + ')')

    fig.update_xaxes(rangeslider_visible=RANGE_SLIDER_VISIBLE)
    fig.update_traces(opacity=.2)
    fig.data[0].opacity = 1
    fig.show()


In [503]:
all_weekly_returns_df = all_weekly_df[risk_tickers]
plot_measures('Weekly Returns', all_weekly_returns_df)

In [509]:
std_columns = [ticker + '_7d_std' for ticker in risk_tickers]
plot_measures('Weekly Standard Deviation', all_weekly_df[std_columns])

In [510]:
sharpe_columns = [ticker + '_sharpe' for ticker in risk_tickers]
plot_measures('Weekly Sharpe Ratio', all_weekly_df[sharpe_columns])

In [511]:
var95_columns = [ticker + '_VaR_95' for ticker in risk_tickers]
plot_measures('Weekly VaR (95%)', all_weekly_df[var95_columns])

In [512]:
beta_columns = [ticker + '_beta' for ticker in risk_tickers]
plot_measures('Weekly Beta', all_weekly_df[beta_columns])

In [513]:
jensen_columns = [ticker + '_jensen' for ticker in risk_tickers]
plot_measures("Weekly Jensen's Alpha", all_weekly_df[jensen_columns])